In [1]:
import random
import numpy as np
import crocoddyl
import keras
from keras.models import load_model
from keras import regularizers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
crocoddyl.switchToNumpyMatrix();

Using TensorFlow backend.
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/a/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py

In [2]:
def iteration_analysis(MAXITER:int =1000):
    """
    Script to store the number of iterations crocoddyl takes to generate a trajectory.
    Here, we generate 10K trajectories and store the number of iterations taken by  crocoddyl 
    for each trajectory
    
    """
    iterations = []
    not_feasible = 0
    
    NUM_TRAJ = 10000


    for _ in range(NUM_TRAJ):
        initial_config = [random.uniform(-2.1, 2.),
                                    random.uniform(-2.1, 2.), 
                                    random.uniform(0, 1)]

        model = crocoddyl.ActionModelUnicycle()

        model.costWeights = np.matrix([1, 0.3]).T

        problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)

        ddp = crocoddyl.SolverDDP(problem)

        done = ddp.solve([], [], MAXITER)
        if not ddp.isFeasible:
            not_feasible += 1

        else:
            iterations.append(ddp.iter)

    max_value = max(iterations)
    min_value = min(iterations)
    avg_value = sum(iterations)//len(iterations)
    print(f"Total trajectories: {NUM_TRAJ}, not feasible trajectories: {not_feasible}\n\n")
    print(f'Maximum iterations allowed : {MAXITER}\n')
    print(f"\nMinimum iterations taken -> {min_value},\
          Maximum iterations taken -> {max_value},\
          Average -> {avg_value} \n")
    print(".............................\n")
    
    return iterations

Group1 = iteration_analysis(MAXITER = 1000)
Group2 = iteration_analysis(MAXITER = 100)
Group3 = iteration_analysis(MAXITER = 500)

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 1000


Minimum iterations taken -> 2,          Maximum iterations taken -> 409,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 100


Minimum iterations taken -> 2,          Maximum iterations taken -> 100,          Average -> 25 

.............................

Total trajectories: 10000, not feasible trajectories: 0


Maximum iterations allowed : 500


Minimum iterations taken -> 2,          Maximum iterations taken -> 470,          Average -> 26 

.............................



In [3]:
def generate_data(ntrajectories: int = 10000,
                  maxiter: int = 100,
                  knots: int = 20,
                  state_weight: float = 1., # can use random.uniform(1, 3)
                  control_weight: float = 0.3, # can use random.uniform(0,1)
                 ):
    """
    @ Arguments:
        ntrajectories : number of trajectories to generate
        maxiter       : number of iterations crocoddyl is supposed to make for each trajectory
        knots         : Knots, e.g T = 20, 30
        
        
    @ Returns:
        initial_states         : matrix of starting points, shape (ntrajectories, 3)....x_data
        optimal trajectories   : matrix of trajectories, shape (ntrajectores, knots).....y_data
                                 Each row of this matrix is:
                                     x1, y1, theta1, v1, w1, x2, y2, theta2, v2, w2, .....so on
        
    """
    starting_configurations = []
    optimal_trajectories = []
    
    for _ in range(ntrajectories):
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]
            
            model = crocoddyl.ActionModelUnicycle()
            model.costWeights = np.matrix([state_weight, control_weight]).T
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * knots, model)
            ddp = crocoddyl.SolverDDP(problem)
            ddp.solve([], [], maxiter)
            
            if ddp.isFeasible:
                state = np.array(ddp.xs)
                control = np.array(ddp.us)

                optimal_trajectory = np.hstack((state[1:,:], control))
                optimal_trajectory = np.ravel(optimal_trajectory)
                optimal_trajectories.append(optimal_trajectory)
                starting_configurations.append(initial_config)
                
    optimal_trajectories = np.array(optimal_trajectories).clip(min=1e-10)
    starting_configurations = np.array(starting_configurations).clip(min=1e-10)         
    return starting_configurations, optimal_trajectories
    

In [ ]:

def train_net(n_hidden: int = 4, save_model: bool = False):
    """
    A generic keras 4 hidden layered neural net with RMSprop as optimizer
    Each Layer has 256 neurons.
    
    """
    x_data, y_data = generate_data()
    
    
    model = keras.models.Sequential()
    
    model.add(Dense(256, input_dim=(x_data.shape[1])))
    model.add(Activation('relu'))
    for _ in range(n_hidden):
        model.add(Dense(256,
                        activation = "tanh",
                        kernel_initializer='random_uniform',
                        kernel_regularizer=regularizers.l2(0.01),
                        activity_regularizer=regularizers.l1(0.01)))            
        model.add(Dropout(0.25))

    model.add(Dense(y_data.shape[1], 
                    activation = 'linear'))        


    rms = optimizers.RMSprop(learning_rate=0.001, rho=0.9)
    model.compile(loss='mean_squared_error',
                  optimizer=rms,
                  metrics=['mean_squared_error', "mean_absolute_error"])
    
    model.fit(x_data, 
                  y_data,
                  epochs = 200,
                  batch_size= 16,
                  verbose = 0
                  )
        
    #x_test, y_test = generate_data(1000) 
    #score = model.evaluate(x_test, y_test, batch_size = 16, use_multiprocessing=True)
    
    if save_model:
        model.save('basic_model.h5')
        
    else: return model    
            
train_net(save_model=True)

In [4]:

def warmstart(n_trajectories: int = 10000, load_net: bool = False):
    if load_net:
        neuralNet = load_model('basic_model.h5')
    else:
        neuralNet = train_net()
        
    print(f" With warmstart, without warmstart ")
    
    for _ in range(n_trajectories):
            
            initial_config = [random.uniform(-2.1, 2.),
                                        random.uniform(-2.1, 2.), 
                                        random.uniform(0, 1)]

            model = crocoddyl.ActionModelUnicycle()
            model2 = crocoddyl.ActionModelUnicycle()
            
            model.costWeights = np.matrix([1., .3]).T
            model2.costWeights = np.matrix([1., .3]).T
    
            
            problem = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model ] * 20, model)
            problem2 = crocoddyl.ShootingProblem(np.matrix(initial_config).T, [ model2 ] * 20, model2)

            
            ddp = crocoddyl.SolverDDP(problem)
            ddp2 = crocoddyl.SolverDDP(problem2)
            
            
            x_data = np.matrix(initial_config).reshape(1, 3)

            prediction = neuralNet.predict(x_data).reshape(20, 5)

            xs = np.matrix(prediction[:, 0:3])

            us = np.matrix(prediction[:, 3:5])
            

            ddp_xs = []
            ddp_xs.append(np.matrix(initial_config).reshape(1, 3).T)
            for _ in  range(xs.shape[0]):
                ddp_xs.append((np.matrix(xs[_]).T))

            ddp_us = []
            for _ in  range(us.shape[0]):
                ddp_us.append(np.matrix(us[_]).T)

            ddp.solve(ddp_xs, ddp_us)
            ddp2.solve()
            print("  ", ddp.iter, "  ", ddp2.iter)
                
    

In [5]:
warmstart(n_trajectories=5000, load_net=True)


 With warmstart, without warmstart 
   10    11
   18    17
   24    26
   55    60
   27    25
   35    28
   47    35
   28    32
   11    11
   34    33
   21    23
   6    8
   65    89
   85    34
   100    93
   7    7
   42    37
   17    21
   19    19
   48    22
   23    20
   25    24
   43    46
   25    26
   4    5
   24    27
   9    10
   19    18
   8    9
   72    43
   43    45
   100    100
   21    26
   30    33
   80    23
   19    16
   54    52
   37    33
   17    16
   27    31
   11    13
   28    27
   8    7
   11    10
   3    3
   16    18
   19    23
   12    12
   9    8
   7    6
   21    23
   18    16
   61    53
   28    24
   30    33
   35    34
   100    85
   11    9
   20    26
   7    6
   16    25
   16    14
   6    5
   32    29
   34    32
   69    76
   13    16
   20    18
   22    26
   17    23
   58    44
   28    32
   41    43
   11    12
   92    48
   24    26
   4    4
   8    9
   65    69
   21    25
   17    15
   71    39
 

   29    33
   5    5
   24    27
   11    13
   64    63
   9    8
   31    27
   18    25
   14    16
   24    21
   5    5
   53    47
   12    12
   36    47
   25    22
   36    26
   31    25
   5    5
   8    8
   9    9
   10    10
   26    25
   9    8
   6    6
   7    8
   21    26
   56    56
   23    22
   34    31
   32    24
   48    34
   12    14
   38    34
   3    3
   23    25
   10    10
   30    29
   29    33
   52    48
   35    39
   21    23
   56    39
   5    6
   18    17
   20    20
   12    11
   22    19
   7    8
   38    41
   38    33
   33    30
   33    35
   25    29
   100    54
   100    45
   7    7
   11    11
   22    18
   29    27
   5    5
   39    36
   31    25
   20    24
   31    31
   8    9
   52    63
   13    13
   20    20
   37    36
   26    25
   29    27
   6    6
   77    63
   11    11
   38    24
   100    32
   5    4
   26    22
   16    16
   37    31
   22    21
   43    38
   57    90
   6    6
   12    11
   7    7
   

   41    48
   56    33
   26    28
   42    44
   64    65
   11    10
   50    47
   26    25
   29    27
   7    5
   18    17
   16    16
   16    15
   17    25
   16    18
   8    8
   7    7
   20    19
   34    39
   16    14
   13    12
   38    49
   20    21
   7    6
   27    28
   11    10
   22    25
   32    34
   100    33
   38    45
   25    24
   6    4
   14    16
   85    29
   15    14
   15    15
   3    3
   10    11
   12    15
   11    13
   20    20
   40    31
   17    20
   11    12
   68    51
   20    17
   12    11
   3    3
   35    35
   6    7
   6    5
   39    37
   6    5
   26    22
   24    28
   10    12
   43    58
   31    27
   10    10
   5    5
   16    18
   30    35
   38    28
   40    38
   14    15
   5    6
   22    22
   65    58
   38    39
   16    19
   27    32
   40    51
   30    31
   32    24
   100    83
   4    4
   48    43
   14    17
   28    36
   32    30
   20    24
   28    32
   35    34
   48    64
   31    32
   1

   100    100
   16    17
   46    22
   21    20
   29    29
   28    31
   25    25
   5    6
   8    8
   57    68
   33    32
   42    37
   25    25
   15    17
   25    25
   25    21
   15    14
   39    41
   40    42
   6    5
   14    15
   26    31
   62    45
   48    34
   54    48
   5    5
   9    8
   24    26
   6    6
   26    28
   21    20
   13    12
   36    22
   71    54
   22    18
   9    10
   36    29
   15    16
   30    27
   14    16
   19    18
   90    38
   8    9
   30    26
   13    12
   19    26
   37    30
   30    29
   9    9
   48    42
   43    45
   65    49
   5    5
   65    59
   24    23
   11    10
   56    74
   10    9
   33    32
   34    30
   21    25
   25    22
   11    14
   18    16
   25    24
   29    21
   7    8
   24    23
   8    8
   6    7
   22    18
   38    35
   5    5
   6    6
   41    49
   25    31
   7    8
   16    25
   6    6
   9    9
   9    10
   33    29
   4    4
   6    5
   5    5
   28    25
   8    8

   11    12
   15    13
   12    11
   84    57
   12    14
   4    5
   12    11
   51    41
   19    17
   4    3
   42    32
   9    9
   23    20
   30    25
   31    28
   45    35
   61    28
   100    100
   13    12
   5    4
   30    34
   19    19
   15    14
   7    6
   20    20
   78    36
   14    15
   21    20
   8    8
   18    20
   8    8
   21    19
   30    34
   12    13
   22    22
   55    41
   6    6
   7    7
   27    31
   28    30
   10    11
   10    10
   37    37
   27    25
   30    27
   82    46
   45    40
   8    9
   50    39
   13    12
   43    42
   35    29
   83    53
   96    48
   28    27
   24    23
   15    15
   53    55
   25    27
   66    56
   10    11
   5    5
   24    21
   28    24
   9    9
   30    33
   13    16
   22    22
   22    20
   100    100
   4    4
   14    13
   29    26
   11    11
   6    6
   7    8
   21    18
   6    5
   56    76
   26    24
   6    6
   100    71
   15    18
   13    16
   75    46
   100   

   23    19
   5    4
   38    29
   21    21
   17    18
   25    25
   36    26
   16    15
   100    100
   6    5
   5    4
   66    81
   7    7
   15    14
   26    26
   17    16
   16    14
   25    27
   4    4
   11    14
   13    13
   10    10
   100    100
   14    13
   26    22
   28    25
   53    47
   6    6
   28    26
   27    27
   27    27
   55    58
   29    31
   9    8
   5    6
   54    29
   21    24
   44    45
   29    34
   92    32
   38    38
   14    17
   18    16
   11    11
   84    84
   46    41
   28    26
   18    17
   71    56
   62    78
   14    14
   31    35
   18    12
   34    31
   34    36
   29    27
   16    14
   78    92
   16    15
   7    8
   49    45
   16    17
   60    54
   15    14
   18    20
   3    4
   51    63
   42    40
   5    4
   15    15
   23    20
   7    7
   72    100
   8    8
   37    27
   14    15
   32    28
   13    12
   100    37
   20    25
   9    8
   24    23
   27    29
   21    21
   43    43
  

   9    8
   11    10
   34    31
   35    30
   51    41
   49    47
   19    22
   27    32
   13    12
   11    11
   7    7
   14    15
   27    26
   28    26
   48    45
   18    18
   24    24
   7    6
   31    30
   53    89
   47    42
   11    14
   27    24
   6    6
   13    13
   24    20
   18    16
   55    49
   29    26
   31    29
   28    27
   23    23
   26    32
   21    19
   6    6
   20    21
   9    8
   27    25
   7    8
   6    5
   7    7
   49    93
   58    58
   20    18
   25    27
   75    63
   26    21
   58    48
   38    39
   25    24
   4    5
   6    5
   45    49
   29    23
   18    14
   33    33
   24    24
   18    19
   11    13
   12    13
   20    19
   13    13
   9    8
   11    10
   18    16
   18    17
   34    34
   19    18
   18    17
   17    19
   21    22
   45    45
   29    26
   12    11
   10    10
   64    68
   4    4
   13    12
   6    7
   40    29
   5    6
   10    10
   5    5
   8    10
   99    100
   46    47
